In [173]:
import pandas as pd
import time

In [174]:
old=pd.read_json('data/old.json')#.reset_index(drop=True)
old.number_of_citations.sum()

1636

Download citations of each paper

In [175]:
UPDATE=False
if UPDATE:
    for r in old['recid']:
        p=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r)
                      ).to_json('data/{}.json'.format(r))
        print(r)
        time.sleep(1)

Download INSPIRE HEP Profile

In [176]:
# TODO: Check if the author has more that 250 records!
exact_author='D.Restrepo.1'
url='http://inspirehep.net/search?p=ea+{}&rg=250&of=recjson'.format(exact_author)
df=pd.read_json(url)
new=df.copy()
df.number_of_citations.sum()

1638

author='restrepo,+d'
df=pd.read_json(
    "http://inspirehep.net/search?ln=en&ln=en&p=a+{}+AND+collection%3Aciteable&rg=250&of=recjson".format(
        author))
new=df.copy()

In [177]:
import sys
ANALYSIS=False
if new['number_of_citations'].sum()>old['number_of_citations'].sum():
    ANALYSIS=True
if not ANALYSIS:
    sys.exit('NO NEW CITES TO ANALYSE!')

In [178]:
old['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]
new['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]

In [179]:
old=old.rename({'number_of_citations':'nc'},axis='columns') ##add other relevant columns for analysis

In [180]:
ANALYSIS

True

In [181]:
new=new.merge(old[['nc','iid']],on='iid',how='left')#[['number_of_citations','nc']]

In [182]:
news=new[new['number_of_citations']>new['nc']]

In [183]:
print('New citations in:')
news['recid']
#['primary_report_number'].apply(lambda l:  [x for x in l if x.find('arXiv')>-1])

New citations in:


1    1746620
6    1679809
Name: recid, dtype: int64

In [185]:
r

1746620

In [193]:
refersto['recid']

0    1759674
Name: recid, dtype: int64

In [192]:
pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'])

array([1759674])

In [199]:
pd.np.setdiff1d(refersto['recid'],[])

array([1759674])

In [200]:
for r in news['recid']:
    #Analysis of each citation
    print('===== NEW CITATION FOR {}====='.format(r))
    oldrefersto=pd.read_json('data/{}.json'.format(r))
    refersto=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r))
    print( news[news['recid']==r]['title'].values[0] )
    print('with')
    print(news[news['recid']==r]['authors'].apply(lambda l: [d['full_name'] for d in l]).values[0])
    print('WAS CITED BY:')
    try:
        newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'])
    except KeyError:
        newrecid=pd.np.setdiff1d(refersto['recid'],[])
    #newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'][:-2])
    for rr in newrecid:
        print('*'*60)
        print( refersto[refersto['recid']==rr]['title'].values[0])
        print( refersto[refersto['recid']==rr]['authors'].apply(lambda l: 
                                            [d['full_name'] for d in l]).values[0])
        print( refersto[refersto['recid']==rr]['primary_report_number'].apply(lambda l: 
                                            [x for x in l if x.find('arXiv')>-1]).values[0] )    
    print('UPDATE CITATIONS (Y)')
    YN=input('Y/N')
    if YN!='N' or YN!='n' or YN!='not' or YN!='Not':
        print('UPDATING JSON DATA CITATION for {}'.format(r))
        refersto.to_json('data/{}.json'.format(r))

===== NEW CITATION FOR 1746620=====
{'title': 'Phenomenological consistency of the singlet-triplet scotogenic model'}
with
['Restrepo, Diego', 'Rivera, Andrés']
WAS CITED BY:
************************************************************
{'title': 'Minimalistic scotogenic scalar dark matter'}
['Ávila, Ivania M.', 'De Romeri, Valentina', 'Duarte, Laura', 'Valle, José W.F.']
['arXiv:1910.08422']
UPDATE CITATIONS (Y)
Y/Ny
UPDATINF JSON DATA CITATION for 1746620
===== NEW CITATION FOR 1679809=====
{'title': 'Bound-state dark matter with Majorana neutrinos'}
with
['Reig, M.', 'Restrepo, D.', 'Valle, J.W. F.', 'Zapata, O.']
WAS CITED BY:
************************************************************
{'title': 'Minimalistic scotogenic scalar dark matter'}
['Ávila, Ivania M.', 'De Romeri, Valentina', 'Duarte, Laura', 'Valle, José W.F.']
['arXiv:1910.08422']
UPDATE CITATIONS (Y)
Y/Ny
UPDATINF JSON DATA CITATION for 1679809


In [201]:
print('UPDATE FULL DATABASE for profiles (N)')
YN=input('Y/N')
if YN=='Y' or YN=='y' or YN=='yes' or YN=='Yes':
    df.to_json('data/old.json')

UPDATE FULL DATABASE for profiles (N)
Y/Ny
